In [ ]:
pip install unidecode

     |████████████████████████████████| 245kB 7.6MB/s 


In [ ]:
!pip install emoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 197 kB 4.2 MB/s 
  Created wheel for emoji: filename=emoji-2.0.0-py3-none-any.whl size=193022 sha256=c5ebe4208c0442d969dd15911f4c5bcc7b1caa3153e1a32fcd755490d65314de
  Stored in directory: /root/.cache/pip/wheels/ec/29/4d/3cfe7452ac7d8d83b1930f8a6205c3c9649b24e80f9029fc38
Successfully built emoji


In [ ]:
!git clone https://github.com/carpedm20/emoji.git
%cd emoji
!pip3 install emoji

Cloning into 'emoji'...
remote: Enumerating objects: 898, done.
remote: Counting objects: 100% (131/131), done.
remote: Compressing objects: 100% (108/108), done.
remote: Total 898 (delta 53), reused 71 (delta 17), pack-reused 767
Receiving objects: 100% (898/898), 1.42 MiB | 17.26 MiB/s, done.
Resolving deltas: 100% (467/467), done.
/content/emoji


In [ ]:
pip install plotly

In [ ]:
import regex as re
import pandas as pd
import numpy as np
import emoji
import plotly.express as px
from collections import Counter
import matplotlib.pyplot as plt
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
%matplotlib inline

Although, the data is ready to use we still need to change the format of the date and time of messages which can be done easily. For this I will define a function that can detect whether each line starts with a date as it states that it is a unique message:

In [ ]:
def startsWithDateAndTime(s):
    pattern = '^([0-9]+)(\/)([0-9]+)(\/)([0-9]+), ([0-9]+):([0-9]+)[ ]?(AM|PM|am|pm)? -' 
    result = re.match(pattern, s)
    if result:
        return True
    return False

In [ ]:
startsWithDateAndTime('7/26/18, 22:51 - Bobby: This message was deleted')

True

Now I will create a function to extract the usernames in the chats as Authors:

In [ ]:
def FindAuthor(s):
  s=s.split(":")
  if len(s)==2:
    return True
  else:
    return False

Now, I will create a function to separate all the information from each other so that we could easily use the information as a pandas dataframe:

In [ ]:
def getDataPoint(line):   
    splitLine = line.split(' - ') 
    dateTime = splitLine[0]
    date, time = dateTime.split(', ') 
    message = ' '.join(splitLine[1:])
    if FindAuthor(message): 
        splitMessage = message.split(': ') 
        author = splitMessage[0] 
        message = ' '.join(splitMessage[1:])
    else:
        author = None
    return date, time, author, message

The code below will help you to get the data, if you are using an IDE or Jupyter notebook or Google Colab on anything, you can use the code below, you just need to make sure that you write the complete path of your dataset if you are not using Colab or Notebook:

In [ ]:
from google.colab import files
uploaded = files.upload()
data = [] # List to keep track of data so it can be used by a Pandas dataframe
parsedData = []
conversation = '/content/WhatsApp Chat with MSRMC Unofficial.txt'
with open(conversation, encoding="utf-8") as fp:
    fp.readline() # Skipping first line of the file because contains information related to something about end-to-end encryption
    messageBuffer = [] 
    date, time, author = None, None, None
    while True:
        line = fp.readline() 
        if not line: 
            break
        line = line.strip() 
        if startsWithDateAndTime(line): 
            if len(messageBuffer) > 0: 
                parsedData.append([date, time, author, ' '.join(messageBuffer)]) 
            messageBuffer.clear() 
            date, time, author, message = getDataPoint(line) 
            messageBuffer.append(message) 
        else:
            messageBuffer.append(line)

Saving WhatsApp Chat with MSRMC Unofficial.txt to WhatsApp Chat with MSRMC Unofficial (1).txt


Now, let’s put the data into a dataframe and have a look at the data:

In [ ]:
df = pd.DataFrame(parsedData, columns=['Date', 'Time', 'Author', 'Message']) # Initialising a pandas Dataframe.
df["Date"] = pd.to_datetime(df["Date"])
df.tail(20)

,Date,Time,Author,Message
20254,2021-05-25,18:24,+91 6364 232 329,Rip
20255,2021-05-25,18:25,+91 81235 47628,Only biochem right??
20256,2021-05-25,18:26,+91 70909 53619,Yeah
20257,2021-05-25,18:26,+91 70909 53619,As of now
20258,2021-05-25,18:26,+91 81235 47628,K thx
20259,2021-05-25,18:26,+91 70909 53619,Only biochem department sent it
20260,2021-05-25,18:26,+91 81235 47628,I 👀
20261,2021-05-25,18:35,+91 98804 89289,Ok here is what we do
20262,2021-05-25,18:36,+91 98804 89289,We do not speak of any tests or anything condu...
20263,2021-05-25,18:36,+91 98804 89289,Cause somone told the biochem teacher today we...


The above dataframe looks good. Now let’s start with our WhatsApp group chat analysis.

To Get All The Authors:

The Authors are representing all the participants of the WhatsApp group, now let’s use how we can extract names of all the authors:

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20274 entries, 0 to 20273
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   Date     20274 non-null  datetime64[ns]
 1   Time     20274 non-null  object        
 2   Author   19792 non-null  object        
 3   Message  20274 non-null  object        
dtypes: datetime64[ns](1), object(3)
memory usage: 633.7+ KB


In [ ]:
df.Author.unique()

array([None, '+91 97443 91319', '+91 70220 78482', '+91 74066 21031',
       '+91 6206 856 451', '+91 99027 63675', '+91 74950 85201',
       'Anup Sinnur MSRMC', '+91 98125 95977', '+91 96202 41599',
       '+91 89019 53005', '+91 82773 78905', '+91 99861 46206',
       '+91 96323 64010', '+91 94497 11461', '+91 83108 76613',
       '+91 97396 25857', '+91 86606 31976', '+91 81689 65042',
       '+91 89041 86587', '+91 78997 73809', '+91 85508 34528',
       '+91 6364 232 329', '+91 86186 44851', '+91 77957 11609',
       '+91 96865 41171', 'Sneha MSRMC', '+91 77958 18337',
       'Kalyan MSRMC', '+91 76196 01077', 'Bhargav V Naik',
       '+91 80882 43371', '+91 73383 64968', '+91 80882 97463',
       '+91 83102 21167', '+91 91130 87875', '+91 74113 81659',
       '+91 91104 02601', '+91 99004 23041', '+91 98706 45422',
       '+91 81689 12746', '+91 88619 45411', '+91 99000 64322',
       '+91 99020 96035', '+91 82960 50540', '+91 96103 41776',
       '+91 99018 89071', '+91 81235 4

WhatsApp Group Chat Analysis: Group Wise Status:-
Now let’s have some analysis by looking at the statistics. I will first create a function which will split the text and other media files from each other including emojis:

In [ ]:
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19792 entries, 3 to 20273
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   Date     19792 non-null  datetime64[ns]
 1   Time     19792 non-null  object        
 2   Author   19792 non-null  object        
 3   Message  19792 non-null  object        
dtypes: datetime64[ns](1), object(3)
memory usage: 773.1+ KB


In [ ]:
total_messages = df.shape[0]
print(total_messages)

19792


In [ ]:
media_messages = df[df['Message'] == '<Media omitted>'].shape[0]
print(media_messages)


1067


In [ ]:
print(df)

            Date   Time            Author  \
3     2020-12-30  20:07   +91 97443 91319   
7     2020-12-30  20:10   +91 70220 78482   
8     2020-12-30  20:11   +91 74066 21031   
9     2020-12-30  20:12   +91 70220 78482   
10    2020-12-30  20:12  +91 6206 856 451   
...          ...    ...               ...   
20268 2021-05-25  19:18   +91 99727 50154   
20269 2021-05-25  20:17   +91 98954 61546   
20270 2021-05-26  08:59   +91 70220 78482   
20272 2021-05-26  09:07   +91 70220 78482   
20273 2021-05-26  11:09   +91 99727 50154   

                                                 Message  
3                     Hi Friends I'm Arunima from Kerala  
7      Guys anyone from Bihar, Jharkhand, West Bengal...  
8          There are many of them And I'm from Bangalore  
9                                                   Cool  
10                                    Rishabh from Bihar  
...                                                  ...  
20268                                         

In [ ]:
import re

## Tokenization

In [ ]:
def split_count(text):

    emoji_list = []
    data = re.findall(r'\\X', text)
    #print(data)
    for word in data:
      emojis = emoji.distinct_emoji_list(word)
      emoji_list.extend([emoji.demojize(is_emoji) for is_emoji in emojis])

    return emoji_list

df["emoji"] = df["Message"].apply(split_count)


In [ ]:
def split_count(text):

    emoji_list = []
    data = re.findall(r'\\X', text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI for char in word):
            emoji_list.append(word)

    return emoji_list

df["emoji"] = df["Message"].apply(split_count)

In [ ]:
df.head(20)

,Date,Time,Author,Message,emoji,urlcount
3,2020-12-30,20:07,+91 97443 91319,Hi Friends I'm Arunima from Kerala,[],0
7,2020-12-30,20:10,+91 70220 78482,"Guys anyone from Bihar, Jharkhand, West Bengal...",[],0
8,2020-12-30,20:11,+91 74066 21031,There are many of them And I'm from Bangalore,[],0
9,2020-12-30,20:12,+91 70220 78482,Cool,[],0
10,2020-12-30,20:12,+91 6206 856 451,Rishabh from Bihar,[],0
11,2020-12-30,20:12,+91 99027 63675,Where are you guys from in bangalore,[],0
12,2020-12-30,20:12,+91 74950 85201,"Hey Guys, I'm Ashish bhanwala from haryana.",[],0
13,2020-12-30,20:12,Anup Sinnur MSRMC,I am Anup from Bangalore,[],0
14,2020-12-30,20:13,+91 74066 21031,Rajajinagar Way?,[],0
16,2020-12-30,20:13,+91 98125 95977,Frm which city.??..as M 2 frm Haryana,[],0


In [ ]:
emojis = sum(df['emoji'].str.len())
print(emojis)

0


In [ ]:
URLPATTERN = r'(https?://\S+)'
df['urlcount'] = df.Message.apply(lambda x: re.findall(URLPATTERN, x)).str.len()

In [ ]:
links = np.sum(df.urlcount)

In [ ]:
print("Data science Community")
print("Messages:",total_messages)
print("Media:",media_messages)
print("Emojis:",emojis)
print("Links:",links)

Data science Community
Messages: 19792
Media: 1067
Emojis: 0
Links: 70


In [ ]:
media_messages_df = df[df['Message'] == '<Media omitted>']

In [ ]:
messages_df = df.drop(media_messages_df.index)

In [ ]:
messages_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18725 entries, 3 to 20273
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Date      18725 non-null  datetime64[ns]
 1   Time      18725 non-null  object        
 2   Author    18725 non-null  object        
 3   Message   18725 non-null  object        
 4   emoji     18725 non-null  object        
 5   urlcount  18725 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 1.0+ MB


## Summerization

Named Entity, Topic Segmentation

In [ ]:
messages_df['Letter_Count'] = messages_df['Message'].apply(lambda s : len(s))
messages_df['Word_Count'] = messages_df['Message'].apply(lambda s : len(s.split(' ')))
messages_df["MessageCount"]=1

In [ ]:
messages_df.tail(20)

,Date,Time,Author,Message,emoji,urlcount,Letter_Count,Word_Count,MessageCount
20253,2021-05-25,18:23,+91 70909 53619,Will let u know when she replies,[],0,32,7,1
20254,2021-05-25,18:24,+91 6364 232 329,Rip,[],0,3,1,1
20255,2021-05-25,18:25,+91 81235 47628,Only biochem right??,[],0,20,3,1
20256,2021-05-25,18:26,+91 70909 53619,Yeah,[],0,4,1,1
20257,2021-05-25,18:26,+91 70909 53619,As of now,[],0,9,3,1
20258,2021-05-25,18:26,+91 81235 47628,K thx,[],0,5,2,1
20259,2021-05-25,18:26,+91 70909 53619,Only biochem department sent it,[],0,31,5,1
20260,2021-05-25,18:26,+91 81235 47628,I 👀,[],0,3,2,1
20261,2021-05-25,18:35,+91 98804 89289,Ok here is what we do,[],0,21,6,1
20262,2021-05-25,18:36,+91 98804 89289,We do not speak of any tests or anything condu...,[],0,78,16,1


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19792 entries, 3 to 20273
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Date      19792 non-null  datetime64[ns]
 1   Time      19792 non-null  object        
 2   Author    19792 non-null  object        
 3   Message   19792 non-null  object        
 4   emoji     19792 non-null  object        
 5   urlcount  19792 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 1.6+ MB


In [ ]:
df.Author.unique()

array(['+91 97443 91319', '+91 70220 78482', '+91 74066 21031',
       '+91 6206 856 451', '+91 99027 63675', '+91 74950 85201',
       'Anup Sinnur MSRMC', '+91 98125 95977', '+91 96202 41599',
       '+91 89019 53005', '+91 82773 78905', '+91 99861 46206',
       '+91 96323 64010', '+91 94497 11461', '+91 83108 76613',
       '+91 97396 25857', '+91 86606 31976', '+91 81689 65042',
       '+91 89041 86587', '+91 78997 73809', '+91 85508 34528',
       '+91 6364 232 329', '+91 86186 44851', '+91 77957 11609',
       '+91 96865 41171', 'Sneha MSRMC', '+91 77958 18337',
       'Kalyan MSRMC', '+91 76196 01077', 'Bhargav V Naik',
       '+91 80882 43371', '+91 73383 64968', '+91 80882 97463',
       '+91 83102 21167', '+91 91130 87875', '+91 74113 81659',
       '+91 91104 02601', '+91 99004 23041', '+91 98706 45422',
       '+91 81689 12746', '+91 88619 45411', '+91 99000 64322',
       '+91 99020 96035', '+91 82960 50540', '+91 96103 41776',
       '+91 99018 89071', '+91 81235 47628',

In [ ]:
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19792 entries, 3 to 20273
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Date      19792 non-null  datetime64[ns]
 1   Time      19792 non-null  object        
 2   Author    19792 non-null  object        
 3   Message   19792 non-null  object        
 4   emoji     19792 non-null  object        
 5   urlcount  19792 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 1.1+ MB


In [ ]:
total_messages = df.shape[0]
print(total_messages)

19792


In [ ]:
media_messages = df[df['Message'] == '<Media omitted>'].shape[0]
print(media_messages)

1067


In [ ]:
def split_count(text):

    emoji_list = []
    data = re.findall(r'\\X', text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI for char in word):
            emoji_list.append(word)

    return emoji_list

df["emoji"] = df["Message"].apply(split_count)

In [ ]:
emojis = sum(df['emoji'].str.len())
print(emojis)

0


## Stemming and Lemmatization

In [ ]:
URLPATTERN = r'(https?://\S+)'
df['urlcount'] = df.Message.apply(lambda x: re.findall(URLPATTERN, x)).str.len()

In [ ]:
links = np.sum(df.urlcount)

In [ ]:
print("Medical Science Community")
print("Messages:",total_messages)
print("Media:",media_messages)
print("Emojis:",emojis)
print("Links:",links)

Medical Science Community
Messages: 19792
Media: 1067
Emojis: 0
Links: 70


In [ ]:
l = ['+91 90367 63634', '+91 97414 64606', '+91 89718 44266']
for i in range(len(l)):
  # Filtering out messages of particular user
  req_df= messages_df[messages_df["Author"] == l[i]]
  # req_df will contain messages of only one particular user
  print(f'Stats of {l[i]} -')
  # shape will print number of rows which indirectly means the number of messages
  print('Messages Sent', req_df.shape[0])
  #Word_Count contains of total words in one message. Sum of all words/ Total Messages will yield words per message
  words_per_message = (np.sum(req_df['Word_Count']))/req_df.shape[0]
  print('Words per message', words_per_message)
  #media conists of media messages
  media = media_messages_df[media_messages_df['Author'] == l[i]].shape[0]
  print('Media Messages Sent', media)
  # emojis conists of total emojis
  emojis = sum(req_df['emoji'].str.len())
  print('Emojis Sent', emojis)
  #links consist of total links
  links = sum(req_df["urlcount"])   
  print('Links Sent', links)   
  print()

Stats of +91 90367 63634 -
Messages Sent 0
Words per message nan
Media Messages Sent 0
Emojis Sent 0
Links Sent 0

Stats of +91 97414 64606 -
Messages Sent 0
Words per message nan
Media Messages Sent 0
Emojis Sent 0
Links Sent 0

Stats of +91 89718 44266 -
Messages Sent 0
Words per message nan
Media Messages Sent 0
Emojis Sent 0
Links Sent 0



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in long_scalars
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
total_emojis_list = list(set([a for b in messages_df.emoji for a in b]))
total_emojis = len(total_emojis_list)
print(total_emojis)

0


In [ ]:
total_emojis_list = list([a for b in messages_df.emoji for a in b])
emoji_dict = dict(Counter(total_emojis_list))
emoji_dict = sorted(emoji_dict.items(), key=lambda x: x[1], reverse=True)
for i in emoji_dict:
  print(i)

In [ ]:
emoji_df = pd.DataFrame(emoji_dict, columns=['emoji', 'count'])
emoji_df

,emoji,count
